### train에 대해서 구하기

In [21]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from cascademodel3 import CascadeCVAE

train_data = torch.load('tensor/train.pt')
val_data   = torch.load('tensor/val.pt')
test_data = torch.load('tensor/test.pt')

C:\Users\User\AppData\Local\Temp\ipykernel_22252\2498871337.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load('tensor/train.pt')
C:\Users\User\AppD

In [22]:
train_mt = train_data['mt']
train_sp = train_data['sp']
train_pt = train_data['pt']
train_op   = train_data['op']

### val에 대해서 구하기

In [23]:
val_mt = val_data['mt']
val_sp = val_data['sp']
val_pt = val_data['pt']
val_op   = val_data['op']

### test에 대해서 구하기

In [24]:
test_mt = test_data['mt']
test_sp = test_data['sp']
test_pt = test_data['pt']
test_op   = test_data['op']

In [25]:


# 1. 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. 데이터 차원 정의 (데이터에 맞게 수정 필요)
# train.ipynb의 출력 예시([1182, 23])를 참고하여 설정하세요.
metal_dim      = train_mt.shape[1]
supporting_dim = train_sp.shape[1]
pretreat_dim   = train_pt.shape[1]
operating_dim  = train_op.shape[1]

# 3. 모델 및 옵티마이저 생성
model = CascadeCVAE(
    metal_dim=metal_dim, 
    supporting_dim=supporting_dim, 
    pretreat_dim=pretreat_dim, 
    operating_dim=operating_dim,
    z_dim=16
).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

Using device: cuda


In [ ]:
num_epochs = 500
history = {'train_loss': [], 'val_loss': []}

for epoch in range(1, num_epochs + 1):
    # --- TRAIN PHASE ---
    model.train()
    train_total_loss = 0.0
    
    for batch in train_loader:
        # 데이터 분리 및 GPU 이동
        mt, sp, pt, op = [d.to(device) for d in batch]
        
        optimizer.zero_grad()
        
        # Forward
        out = model(mt, sp, pt, op)
        
        # Loss 계산 (Stage 1의 7번 인덱스 강조 포함)
        loss, _ = model.compute_loss_all_loss3(
            out, mt, sp, pt,
            beta=1.0, 
            metal_focus_idx=7, 
            metal_focus_weight=2.0 # 강조를 위해 가중치 조절 가능
        )
        
        # Backward & Update
        loss.backward()
        optimizer.step()
        
        train_total_loss += loss.item()
    
    avg_train_loss = train_total_loss / len(train_loader)
    history['train_loss'].append(avg_train_loss)

    # --- VALIDATION PHASE ---
    model.eval()
    val_total_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            mt, sp, pt, op = [d.to(device) for d in batch]
            out = model(mt, sp, pt, op)
            loss, _ = model.compute_loss_all_loss3(out, mt, sp, pt)
            val_total_loss += loss.item()
    
    avg_val_loss = val_total_loss / len(val_loader)
    history['val_loss'].append(avg_val_loss)

    # 10 Epoch마다 진행 상황 출력
    if epoch % 20 == 0:
        print(f"Epoch [{epoch}/{num_epochs}] | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

print("Training Complete!")

Epoch [10/500] | Train Loss: 0.2041 | Val Loss: 0.1294
Epoch [20/500] | Train Loss: 0.1882 | Val Loss: 0.1159
Epoch [30/500] | Train Loss: 0.1710 | Val Loss: 0.1043
Epoch [40/500] | Train Loss: 0.1622 | Val Loss: 0.1005
Epoch [50/500] | Train Loss: 0.1563 | Val Loss: 0.0963
Epoch [60/500] | Train Loss: 0.1527 | Val Loss: 0.0923
Epoch [70/500] | Train Loss: 0.1489 | Val Loss: 0.0907
Epoch [80/500] | Train Loss: 0.1452 | Val Loss: 0.0906
Epoch [90/500] | Train Loss: 0.1432 | Val Loss: 0.0869
Epoch [100/500] | Train Loss: 0.1404 | Val Loss: 0.0863
Epoch [110/500] | Train Loss: 0.1382 | Val Loss: 0.0841
Epoch [120/500] | Train Loss: 0.1362 | Val Loss: 0.0835
Epoch [130/500] | Train Loss: 0.1343 | Val Loss: 0.0824
Epoch [140/500] | Train Loss: 0.1319 | Val Loss: 0.0804
Epoch [150/500] | Train Loss: 0.1292 | Val Loss: 0.0799
Epoch [160/500] | Train Loss: 0.1279 | Val Loss: 0.0800
Epoch [170/500] | Train Loss: 0.1251 | Val Loss: 0.0765
Epoch [180/500] | Train Loss: 0.1245 | Val Loss: 0.0794
E

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()